## Исследование надёжности заёмщиков

Имеются данные о некоторых клиентах банка, включающие в себя образование, наличие детей, возраст, доход, семейный статус, пол, общий стаж.

По этим данным необходимо выявить, что влияет на возврат клиентом кредита в срок.

### 1. Откроем файл с данными и изучим общую информацию.

In [123]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [124]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Таблица содержит 21525 записей. В столбцах присутствует три типа данных: int, float и object (строка). Также видно, что в столбцах days_employed и total_income присутствуют пропуски данных.

### 2. Предобработка данных

### Обработка пропусков

Рассмотри столбец days_employed. Предположим, что значения этого столбца - это трудовой стаж в днях. Проверим сначала, есть ли среди положительных значений числа, похожие на реальность. Допустим, максимальный трудовой стаж у человека может составлять 80 лет или 29200 дней. Проверим, существуют ли в этом столбце положительне числа меньше этого значения.

In [100]:
data[(data['days_employed'] < 29200) & (data['days_employed'] > 0)]['days_employed'].count()

0

Такие значения отсутствуют. Это означает, что к ним в дальнейшем обращаться не будем.

Теперь предположим, что знак "-" означет тире и эти числа более близки к реальности. Проверим то же самое.

In [101]:
print(data[(data['days_employed'] > -29200) & (data['days_employed'] < 0)]['days_employed'].count())
print(data[(data['days_employed'] > -29200) & (data['days_employed'] < 0)]['days_employed'].min() / 365)

15906
-50.38068465909146


Получили, что максимальный трудовой стаж составляет 50 лет. Это вполне соответствует действительности. 

Заменим пропущенные данные средним арифметическим по всем отрицательным числам.

In [102]:
mean_minus = data[data['days_employed'] < 0]['days_employed'].mean() #среднее всех отрицательных чисел
data['days_employed'] = data['days_employed'].fillna(mean_minus)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Для столбца total_income поступим следующим образом: заменим пустые значения на медианный доход в зависимости от типа занятости.

In [103]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [104]:

data_grouped = data.groupby('income_type')
data['total_income'] = data_grouped['total_income'].apply(lambda x: x.fillna(x.median()))
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Были просмотрены два столбца, имеющие пропуски данных. 

Для столбца days_employed было выдвинуто предположение, что "-" не означает отрицательное число, т.к. значения по модулю вполне соответствуют общему стажу в днях. А положительные числа - явно больше, чем возможный трудовой стаж. Была проведена замена NaN значений на среднее арифметическое всех отрицательных чисел.

В столбце total_income пустые значения скорее всего возникили из-за сбоев в выгрузке данных, т.к. обычно при получении кредита этот параметр является важным для положительного решения. Такие значения не желательно обнулять, так как они могут быть важны при дальнейшем исследовании. Здесь все исследуемые были разделены на группы по столбцу income_type. У каждой из 8 групп были посчитаны медианы дохода. Пустые значения были заменены полученными результатами.

### Замена типа данных

Заменим вщественный тип данных в столбце total_income на целочисленный и переведем все данные столбца education к нижнему регистру.

In [105]:
data['total_income'] = data['total_income'].astype('int')
data['education'] = data['education'].str.lower()
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Для изменения типов данных был выбран метод astype(), т.к. все данные изначально находились в численном виде float и ошибок преобразования не предвиделось.

### Обработка дубликатов

Посчитаем общее количество дубликатов во всей таблице в процентном соотношении и удалим дубли.

In [106]:
data_duplicated = data.duplicated().value_counts()
duplicates_count = data_duplicated[True] # количество дубликатов
all_data_len = len(data)

try:
    duplicated_ratio = duplicates_count / all_data_len
except:
    print('Ошибка! Деление на ноль, пустая таблица!')
else:
    print('Количество дубликатов в таблице составляет: {:.2%}'.format(duplicated_ratio))

data = data.drop_duplicates().reset_index(drop=True)
data.info()

Количество дубликатов в таблице составляет: 0.33%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Еще раз проверим наличие дубликатов.

In [107]:
print(data.duplicated().value_counts())

False    21454
dtype: int64


Дубликатов не осталось

### Вывод

Для поиска дубликатов и их количества был выбран метод duplicated().value_counts(). Был выполнен поиск дубликатов по совпадению всей строки. Таких оказалось 0.33% от общего количества данных. Скорее всего, есть небольшая вероятность того, что некоторые удаленные дубликаты на самом деле - разные значения. Но, посчитав количество таких строк, можно не учитывать их влияние на общий результат.

Наличие дубликатов может быть обусловлено программными сбоями при выгрузке таблицы из базы данных.

### Лемматизация

Выделим леммы в значениях столбца purpose. Определим все уникальные значения этого столбца.

In [108]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_row(row):
    lemmas = ' '.join(m.lemmatize(row)) # создаем строку из списка
    return lemmas

data['lemmas_col'] = data['purpose'].apply(lemmatize_row) # столбец с лемматизированными строками
data['lemmas_col'].unique()


    

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

Можно выделить несколько категорий, на что берется кредит: жилье, автомобиль, образование, свадьба, недвижимость. Создадим отдельный столбец cr_goal в таблице data, описывающий назначение кредита одним словом.

In [109]:
def credit_goal(row):
    
    if 'недвижимость' in row or 'жилье' in row:
            return 'жилье'
    if 'автомобиль' in row:
            return 'авто'
    if 'образование' in row:
            return 'учеба'
    return 'мероприятие'

data['cr_goal'] = data['lemmas_col'].apply(credit_goal)
data.head(20)
        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_col,cr_goal
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,авто
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,учеба
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,мероприятие
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,жилье
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,жилье
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,учеба
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,мероприятие
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,жилье


### Вывод

После проведения процесса лемматизации было выявлено, что всех заемщиков можно разделить на 5 категорий. Был создан отдельный столбец, отвечающий на вопрос о цели кредита, одним словом.

### Категоризация данных

Разделим клиентов на категории по количеству детей и уровню дохода. 

Начнем с детей.
Создадим столбец child_status, в котором отсутствие детей у клиента будет значить "бездетный", 1 - 2 ребенка - "есть дети", 3 и более - многодетный.

In [110]:
def child_count(row):
    if row == 0:
        return 'бездетный'
    if 1 <= row < 3:
        return 'есть дети'
    return 'многодетный'

data['child_status'] = data['children'].apply(child_count)
data.groupby('child_status')['child_status'].count() # узнаем, как разделились категории

child_status
бездетный      14091
есть дети       6860
многодетный      503
Name: child_status, dtype: int64

Теперь разделим клиентов банка по уровню дохода. Создадим столбец earn_type, в котором будем считать, что доход: меньше 50.000 - низкий, 50.000-100.000 - средний, больше 100.000 - высокий.

In [111]:
def earn(row):
    if row <= 50000:
        return 'низкий'
    if 50000 < row <= 100000:
        return 'средний'
    return 'высокий'

data['earn_type'] = data['total_income'].apply(earn)
data[['total_income', 'earn_type']].head(30)
print(data.groupby('earn_type')['earn_type'].count()) # узнаем, как разделились категории
data.head()        

earn_type
высокий    16991
низкий       372
средний     4091
Name: earn_type, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_col,cr_goal,child_status,earn_type
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье,есть дети,высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,авто,есть дети,высокий
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье,бездетный,высокий
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,учеба,многодетный,высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,мероприятие,бездетный,высокий


### Вывод

Выше была проведена категоризация клиентов по двум параметрам: количество детей и доход. Каждому клиенту была присвоена категория в зависимости от количества детей и суммы дохода. Именно эти параметры были выбраны, т.к. по остальным категории уже существуют. 

### 3. Определение зависимостей между наличием детей и возвратом кредита в срок.

Составим сводную таблицу, в которой в столбцах расположим количество задолженностей, а в строках - количество детей. 

In [113]:
# исправленный вариант

data_children_debt = data[['debt','child_status']] #создадим таблицу, состоящую из двух столбцов

# создадим сводную таблицу по группам
data_children_debt_pivot = data_children_debt.pivot_table(index='child_status', values='debt', aggfunc='sum')

# добавим в сводную таблицу столбец с количеством клиентов каждой категории
data_children_debt_pivot['count'] = data.groupby('child_status')['child_status'].count()

# создадим столбец с процентным соотношением должников по каждой группе
data_children_debt_pivot['ratio'] = (data_children_debt_pivot['debt'] / data_children_debt_pivot['count']) * 100
data_children_debt_pivot


,debt,count,ratio
child_status,,,
бездетный,1063,14091,7.543822
есть дети,638,6860,9.300292
многодетный,40,503,7.952286


### Вывод

Количество бездетных и многодетных клиентов, имевших задолженности, распределилось примерно одинаково - около 8%. Те, у кого есть дети, в 9% случаев оказывались должниками. 

### 4. Определение зависимостей между семейным положением и возвратом кредита в срок.

Составим аналогичную таблицу.

In [115]:
# исправленный вариант

data_fs_debt = data[['debt','family_status']] #создадим таблицу, состоящую из двух столбцов

# создадим сводную таблицу по группам
data_fs_debt_pivot = data_fs_debt.pivot_table(index='family_status', values='debt', aggfunc='sum')

# добавим в сводную таблицу столбец с количеством клиентов каждой категории
data_fs_debt_pivot['count'] = data.groupby('family_status')['family_status'].count()

# создадим столбец с процентным соотношением должников по каждой группе
data_fs_debt_pivot['ratio'] = (data_fs_debt_pivot['debt'] / data_fs_debt_pivot['count']) * 100
data_fs_debt_pivot


,debt,count,ratio
family_status,,,
Не женат / не замужем,274,2810,9.750890
в разводе,85,1195,7.112971
вдовец / вдова,63,959,6.569343
гражданский брак,388,4151,9.347145
женат / замужем,931,12339,7.545182


### Вывод

Здесь клиенты разделились на две категории. Те, кто был женат или сейчас находится в официальном браке, имеют процент задолженностей около 7%. Клиенты, состоящие в гражданском браке, или не женаты (не замужем) - около 9%.  

### 5. Определение зависимости между уровнем дохода и возвратом кредита в срок.

Поступим аналогичным образом.

In [117]:
# исправленный вариант

data_et_debt = data[['debt','earn_type']] # earn_type - столбец, сгруппированный заранее по величине дохода

# создадим сводную таблицу по группам
data_et_debt_pivot = data_et_debt.pivot_table(index='earn_type', values='debt', aggfunc='sum')

# добавим в сводную таблицу столбец с количеством клиентов каждой категории
data_et_debt_pivot['count'] = data.groupby('earn_type')['earn_type'].count()

# создадим столбец с процентным соотношением должников по каждой группе
data_et_debt_pivot['ratio'] = (data_et_debt_pivot['debt'] / data_et_debt_pivot['count']) * 100
data_et_debt_pivot

,debt,count,ratio
earn_type,,,
высокий,1387,16991,8.163145
низкий,23,372,6.182796
средний,331,4091,8.090931


### Вывод

Клиенты с низким уровнем дохода - самые дисциплинированые (6% имели задолженности). Высокий и средний уровень дохода чаще позволял клиентам иметь долги (таковых 8%).

### 6. Определение влияния цели кредита на его возврат в срок

Выполним операции, аналогичные предыдущим.

In [119]:
# исправленный вариант

data_cg_debt = data[['debt','cr_goal']] # cr_goal - столбец, сгруппированный заранее по цели кредита

# создадим сводную таблицу по группам
data_cg_debt_pivot = data_cg_debt.pivot_table(index='cr_goal', values='debt', aggfunc='sum')

# добавим в сводную таблицу столбец с количеством клиентов каждой категории
data_cg_debt_pivot['count'] = data.groupby('cr_goal')['cr_goal'].count()

# создадим столбец с процентным соотношением должников по каждой группе
data_cg_debt_pivot['ratio'] = (data_cg_debt_pivot['debt'] / data_cg_debt_pivot['count']) * 100
data_cg_debt_pivot

,debt,count,ratio
cr_goal,,,
авто,403,4306,9.359034
жилье,782,10811,7.233373
мероприятие,186,2324,8.003442
учеба,370,4013,9.220035


### Вывод

Самые надежные заемщики - те, кто планирует взять кредит на жилье (7% задолженностей). Около 8% тех, кто хочет сыграть свадьбу, имели ранее задолженности. Доля имевших долги клиентов, собирающихся купить автомобиль или оплатить учебу примерно, примерно одинакова и составляет примерно 9%.

### 7. Общий вывод

В проведенном исследовании отражены некоторые закономерности, которые могут повлиять на решение банка о выдаче кредита. Было выявлено влияние различных факторов на имевшиеся у клентов задоложенности, а именно:
 

- наличие детей влияет на задолженности. Самыми ненадежными плательщиками оказались те, у кого есть 1-2 ребенка. Их количество составило 9,3%;
- семейное положение также повлияло на факты наличия задолженности. Самыми ненадежными категориями являются "не женат/не замужем" и "гражданский брак". Среди них клиентов с долгами около 9,5 %. Клиентов с задолженностями, которые состоят (или ранее состояли) в официальном браке около 7%;
- те, чей уровень дохода расценивается как высокий или средний, оказывались в должниках чаще, чем клиенты с низким доходом (8% против 6%).
- Те, кто собирается приобрести жилье, наиболее ответственно относятся к выплатам кредитов. Всего 7% из них имели ранее задолженности.

